In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta.tables import *
from delta import *
import pyspark.sql.functions as PSF
import psycopg2
import pandas as pd
import os

In [8]:
env_vars = !cat Script/.env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value

/Users/renabouj/Downloads/postgresql-42.3.5.jar


In [9]:
builder = pyspark.sql.SparkSession.builder.appName("PySpark_Postgres_test").config("spark.jars", os.environ.get('SPARKJAR')).config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog").master("local").appName("PySpark_Postgres_test")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

22/05/16 17:25:10 WARN Utils: Your hostname, MacBook-Pro-de-Renato.local resolves to a loopback address: 127.0.0.1; using 10.0.0.102 instead (on interface en0)
22/05/16 17:25:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/renabouj/Codes/tendenciasSI/env/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/renabouj/.ivy2/cache
The jars for the packages stored in: /Users/renabouj/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4c4841db-1253-4d00-a36c-25d1e103bc49;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.2.1 in central
	found io.delta#delta-storage;1.2.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 216ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-core_2.12;1.2.1 from central in [default]
	io.delta#delta-storage;1.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnld

In [10]:
dataf = spark.read.format("jdbc").option("url", os.environ.get('DB_CONNECTION_STRING')) \
    .option("driver", "org.postgresql.Driver").option("dbtable", "sales2") \
    .option("user", os.environ.get('USER')).option("password", os.environ.get('PASSWORD')).load()

In [12]:
dataf.where(dataf.Order_ID == 451691138).show()

+--------------------+---------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|              Region|  Country|Item_Type|Sales_Channel|Order_Priority|Order_Date| Order_ID|Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|
+--------------------+---------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|Australia|     Meat|       Online|             C|  4/4/2011|451691138|5/23/2011|      3300|       500|      364|      1814127|   1568167|      245960|
+--------------------+---------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+



In [3]:
dataf = spark.read.format("csv").option("delimiter", ",").option("header", "true").option("inferSchema", "true") \
        .load("data/200ksales.csv")
dataf.show()

+--------------------+--------------------+-------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|             Country|    Item_Type|Sales_Channel|Order_Priority|Order_Date| Order_ID| Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|
+--------------------+--------------------+-------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|           Australia|         Meat|       Online|             C|  4/4/2011|451691138| 5/23/2011|      4300|       421|      364|      1814127|   1568167|      245960|
|                Asia|          Tajikistan|Personal Care|       Online|             L| 7/12/2018|144177377|  8/1/2018|      4145|        81|       56|       338770|    234897|      103873|
|  Sub-Saharan Africa|          Mozambique|    Cosmetic

In [4]:
dataf.columns

['Region',
 'Country',
 'Item_Type',
 'Sales_Channel',
 'Order_Priority',
 'Order_Date',
 'Order_ID',
 'Ship_Date',
 'Units_Sold',
 'Unit_Price',
 'Unit_Cost',
 'Total_Revenue',
 'Total_Cost',
 'Total_Profit']

In [5]:
dataf.count()

254160

In [17]:
lista_de_colunas_inteiro = ["Units_Sold", "Unit_Price", "Unit_Cost", "Total_Revenue", "Total_Cost", "Total_Profit"]

In [27]:
def convert_data_types(df):
    df = df.withColumn("Units_Sold", df.Units_Sold.cast('bigint'))
    df = df.withColumn("Unit_Price", df.Unit_Price.cast('bigint'))
    df = df.withColumn("Unit_Cost", df.Unit_Cost.cast('bigint'))
    df = df.withColumn("Total_Revenue", df.Total_Revenue.cast('bigint'))
    df = df.withColumn("Total_Cost", df.Total_Cost.cast('bigint'))
    df = df.withColumn("Total_Profit", df.Total_Profit.cast('bigint'))
    return df

In [9]:
def convert_data_types(df, columns_list):
    for column_name in columns_list:
        df.select(PSF.col(column_name).cast('bigint').alias(column_name))
    return df

In [10]:
convert_data_types(dataf, lista_de_colunas_inteiro)

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Sales_Channel: string (nullable = true)
 |-- Order_Priority: string (nullable = true)
 |-- Order_Date: string (nullable = true)
 |-- Order_ID: integer (nullable = true)
 |-- Ship_Date: string (nullable = true)
 |-- Units_Sold: integer (nullable = true)
 |-- Unit_Price: integer (nullable = true)
 |-- Unit_Cost: integer (nullable = true)
 |-- Total_Revenue: integer (nullable = true)
 |-- Total_Cost: integer (nullable = true)
 |-- Total_Profit: integer (nullable = true)

None


DataFrame[Region: string, Country: string, Item_Type: string, Sales_Channel: string, Order_Priority: string, Order_Date: string, Order_ID: int, Ship_Date: string, Units_Sold: int, Unit_Price: int, Unit_Cost: int, Total_Revenue: int, Total_Cost: int, Total_Profit: int]

In [12]:
dataf.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Sales_Channel: string (nullable = true)
 |-- Order_Priority: string (nullable = true)
 |-- Order_Date: string (nullable = true)
 |-- Order_ID: integer (nullable = true)
 |-- Ship_Date: string (nullable = true)
 |-- Units_Sold: integer (nullable = true)
 |-- Unit_Price: integer (nullable = true)
 |-- Unit_Cost: integer (nullable = true)
 |-- Total_Revenue: integer (nullable = true)
 |-- Total_Cost: integer (nullable = true)
 |-- Total_Profit: integer (nullable = true)



In [14]:
%timeit
def save_data_to_database(df, dbname: str, overwrite: bool):
    if overwrite:
        df.write.format("jdbc").option("driver", "org.postgresql.Driver").option("url", os.environ.get('DB_CONNECTION_STRING')) \
        .option("dbtable", dbname).option("user", os.environ.get('USER')).option("password", os.environ.get('PASSWORD')) \
        .mode("overwrite").save()
        
        return print("Overwrite done!")
    
    else:
        df.write.format("jdbc").option("driver", "org.postgresql.Driver").option("url", os.environ.get('DB_CONNECTION_STRING')) \
        .option("dbtable", dbname).option("user", os.environ.get('USER')).option("password", os.environ.get('PASSWORD')).save()
        
        return print("Saved without overwrite!")
        
save_data_to_database(dataf, "sales", True)

Overwrite done!


In [14]:
spa = spark.read.format("jdbc").option("url", os.environ.get('DB_CONNECTION_STRING')) \
    .option("driver", "org.postgresql.Driver").option("dbtable", "sales3") \
    .option("user", os.environ.get('USER')).option("password", os.environ.get('PASSWORD')).load()

In [15]:
spa.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Sales_Channel: string (nullable = true)
 |-- Order_Priority: string (nullable = true)
 |-- Order_Date: string (nullable = true)
 |-- Order_ID: string (nullable = true)
 |-- Ship_Date: string (nullable = true)
 |-- Units_Sold: string (nullable = true)
 |-- Unit_Price: string (nullable = true)
 |-- Unit_Cost: string (nullable = true)
 |-- Total_Revenue: string (nullable = true)
 |-- Total_Cost: string (nullable = true)
 |-- Total_Profit: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)



In [16]:
spa.count()

254160